In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.storage import InMemoryStore
from dotenv import load_dotenv

import os
import gc

In [ ]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Embedding model and LLM

In [ ]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=200)

# Load PDF

In [ ]:
pdf_link = "os-sertoes.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()
len(pages)

# Splitter

In [ ]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

# Storage

In [ ]:
store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings_model, persist_directory="parentChallenge_db")

# Parent Retriever

In [ ]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

parent_document_retriever.add_documents(pages, ids=None)

# Prompt

In [ ]:
TEMPLATE = """
Você é um assistente especializado em literatura brasileira. 
Responda utilizando **apenas** o contexto fornecido. 

Regras:
- Se a resposta não estiver claramente presente no contexto, responda: "Não encontrei informações suficientes no contexto para responder."
- Não utilize conhecimento externo.
- Seja conciso, objetivo e responda em português.
- Quando citar algo do contexto, mantenha fidelidade ao texto.

Query: {question}
Contexto:{context}
"""

rag_promopt = ChatPromptTemplate.from_template(TEMPLATE)  

In [ ]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})
output_parser = StrOutputParser()

# Chain

In [ ]:
chain = setup_retrieval | rag_promopt | llm | output_parser

# Questions

In [ ]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

# Response

In [ ]:
def answer_question(question: str):
    return chain.invoke(question)

In [ ]:
for index, question in enumerate(questions):
    resposta = answer_question(question)
    print({"numero": index, "pergunta": question, "resposta": resposta})